climate = open("/Users/shiqinchoo/Desktop/combined/climate.txt","r")
climate = climate.read()

bio = open("/Users/shiqinchoo/Desktop/combined/bio.txt","r")
bio = bio.read()

connect = open("/Users/shiqinchoo/Desktop/combined/connect.txt","r")
connect = connect.read()

trade = open("/Users/shiqinchoo/Desktop/combined/trade.txt","r")
trade = trade.read()

jobs = open("/Users/shiqinchoo/Desktop/combined/jobs.txt","r")
jobs = jobs.read()

social = open("/Users/shiqinchoo/Desktop/combined/social.txt","r")
social = social.read()

In [1]:
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [2]:
import pandas as pd
import os
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shiqinchoo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shiqinchoo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [4]:
import random
#get tokens
text_data=[]
with open("/Users/shiqinchoo/Desktop/combined/climate.txt","r") as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['willis', 'tower', 'watson', 'former', 'school', 'public', 'policy', 'tsinghua']
['vietnam', 'national', 'university', 'vrije', 'universiteit', 'amsterdam', 'willis', 'tower']
['executive', 'summary']
['level', 'region', 'across', 'world', 'capital', 'country', 'adaptation', 'manage', 'risk']
[]
[]
['attempt', 'qualitatively', 'extent', 'possible', 'quantitatively']
[]
[]
['geological', 'journal', 'geophysical', 'research', 'march', 'volume', 'damon', 'matthew', 'susan']
['system']
['climate', 'change', 'cumulative', 'hurricane', 'rainfall', 'rates', 'average', 'intensity', 'proportion', 'storm', 'reach', 'category']
['recognition', 'could', 'happen', 'quickly', 'possibility', 'cascade', 'consequence']
['penetration', 'average', 'probability', 'person', 'living', 'region', 'experience']
['impact', 'agricultural', 'yield', 'country', 'since', 'agmip', 'climate', 'model', 'project', 'agricultural', 'yield']
[]
[]
['physical', 'supply', 'chains', 'currently', 'place', 'plan', 'future']
[

In [5]:
import gensim
from gensim.utils import simple_preprocess
# Build the bigram and trigram models
bigram = gensim.models.Phrases(text_data, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[text_data], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(text_data)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('/opt/anaconda3/lib/python3.7/site-packages/en_core_web_lg/en_core_web_lg-2.2.5/')

In [8]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [9]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:10])

[[(0, 1), (1, 1), (2, 1), (3, 1)], [], [(4, 1), (5, 1)], [(6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)], [], [], [(12, 1), (13, 1), (14, 1), (15, 1), (16, 1)], [], [], [(17, 1)]]


In [47]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=9, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [48]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.072*"volume" + 0.049*"climate" + 0.049*"issue" + 0.049*"number" + '
  '0.026*"country" + 0.026*"impact" + 0.026*"socioeconomic" + '
  '0.026*"integrate" + 0.026*"effective" + 0.026*"response"'),
 (1,
  '0.044*"flooding" + 0.030*"design" + 0.030*"increase" + 0.016*"system" + '
  '0.016*"ability" + 0.016*"surge" + 0.016*"water" + 0.016*"pluvial" + '
  '0.016*"flash" + 0.016*"tidal"'),
 (2,
  '0.035*"would" + 0.035*"degree" + 0.018*"current" + 0.018*"could" + '
  '0.018*"hazard" + 0.018*"penetration" + 0.018*"widespread" + 0.018*"trigger" '
  '+ 0.018*"internal" + 0.018*"displacement"'),
 (3,
  '0.046*"change" + 0.031*"energy" + 0.017*"system" + 0.017*"warming" + '
  '0.017*"independently" + 0.017*"water" + 0.017*"scenario" + 0.017*"grapple" '
  '+ 0.017*"quality" + 0.017*"trap"'),
 (4,
  '0.054*"climate" + 0.039*"impact" + 0.039*"change" + 0.030*"yield" + '
  '0.030*"agricultural" + 0.016*"system" + 0.016*"reach" + 0.016*"hurricane" + '
  '0.016*"model" + 0.016*"storm"'),
 (5,


In [49]:
#climate
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.111244 -0.060897       1        1  15.971324
4      0.101381 -0.102550       2        1  15.930038
3      0.054014 -0.048623       3        1  14.567926
7      0.095818  0.140786       4        1  13.606369
2     -0.102499  0.049081       5        1  12.287268
5     -0.079373  0.030051       6        1   8.121760
0      0.056015  0.044582       7        1   7.370347
8      0.020538 -0.075003       8        1   6.708526
6     -0.034650  0.022574       9        1   5.436431, topic_info=    Category      Freq           Term     Total  loglift  logprob
19   Default  4.000000        average  4.000000  30.0000  30.0000
17   Default  3.000000         volume  3.000000  29.0000  29.0000
22   Default  6.000000        climate  6.000000  28.0000  28.0000
9    Default  2.000000         region  2.000000  27.0000  27.0000
52   Default  2.000000       physical  2.000000  26.0000  26.0000
21   Default  4.000000         change  4.000000  25.0000  25.0000
106  Default  2.000000      condition  2.000000  24.0000  24.0000
108  Default  2.000000          issue  2.000000  23.0000  23.0000
116  Default  3.000000          today  3.000000  22.0000  22.0000
112  Default  1.000000      extremely  1.000000  21.0000  21.0000
179  Default  1.000000         period  1.000000  20.0000  20.0000
178  Default  1.000000         define  1.000000  19.0000  19.0000
180  Default  1.000000          state  1.000000  18.0000  18.0000
138  Default  2.000000         expose  2.000000  17.0000  17.0000
118  Default  1.000000         number  1.000000  16.0000  16.0000
45   Default  3.000000         impact  3.000000  15.0000  15.0000
56   Default  1.000000         budget  1.000000  14.0000  14.0000
57   Default  1.000000         carbon  1.000000  13.0000  13.0000
130  Default  2.000000       flooding  2.000000  12.0000  12.0000
59   Default  2.000000         degree  2.000000  11.0000  11.0000
69   Default  3.000000          would  3.000000  10.0000  10.0000
189  Default  1.000000  environmental  1.000000   9.0000   9.0000
7    Default  2.000000          level  2.000000   8.0000   8.0000
6    Default  1.000000     adaptation  1.000000   7.0000   7.0000
124  Default  1.000000       decrease  1.000000   6.0000   6.0000
173  Default  1.000000           cool  1.000000   5.0000   5.0000
63   Default  2.000000        current  2.000000   4.0000   4.0000
197  Default  1.000000    perspective  1.000000   3.0000   3.0000
199  Default  1.000000   viticultural  1.000000   2.0000   2.0000
8    Default  1.000000         manage  1.000000   1.0000   1.0000
..       ...       ...            ...       ...      ...      ...
139   Topic9  0.050017      effective  1.091904  -0.1713  -5.7871
142   Topic9  0.050017  socioeconomic  1.091904  -0.1713  -5.7871
141   Topic9  0.050017       response  1.091904  -0.1713  -5.7871
140   Topic9  0.050017      integrate  1.091904  -0.1713  -5.7871
190   Topic9  0.050017         letter  1.119708  -0.1964  -5.7871
191   Topic9  0.050017       research  1.119708  -0.1964  -5.7871
194   Topic9  0.050017         remain  1.119709  -0.1964  -5.7871
195   Topic9  0.050017          track  1.119709  -0.1964  -5.7871
61    Topic9  0.050017      threshold  1.119709  -0.1964  -5.7871
67    Topic9  0.050017        prudent  1.119709  -0.1964  -5.7871
58    Topic9  0.050017        celsius  1.119709  -0.1964  -5.7871
66    Topic9  0.050017     management  1.119709  -0.1964  -5.7871
68    Topic9  0.050017        suggest  1.119709  -0.1964  -5.7871
17    Topic9  0.053697         volume  3.361050  -1.2246  -5.7161
18    Topic9  0.050018         system  3.697194  -1.3909  -5.7871
19    Topic9  0.050018        average  4.320482  -1.5467  -5.7871
118   Topic9  0.050018         number  1.602888  -0.5551  -5.7871
183   Topic9  0.050018          study  1.253651  -0.3094  -5.7871
22    Topic9  0.050018        climate  6.182470  -1.9050  -5.78

In [14]:
import random
#get tokens
text_data2=[]
with open("/Users/shiqinchoo/Desktop/combined/bio.txt","r") as g:
    for line in g:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data2.append(tokens)

['mckinsey', 'global', 'network', 'partner', 'industry', 'management', 'expert']
['fueling', 'innovation', 'could', 'significant', 'impact']
['third', 'total', 'impact', 'could', 'hinge', 'consumer', 'societal', 'regulatory']
['technology', 'living', 'organism', 'computational', 'process']
['source', 'mckinsey', 'global', 'institute', 'analysis']
['single', 'imaging', 'tool', 'single', 'ribonucleic', 'sequence']
['relatively', 'static']
['0.001', 'communications']
['development', 'improvement', 'development', 'extraction', 'improve']
[]
[]
['value', 'proposition', 'business', 'model', 'market', 'operational', 'scalability']
['right', 'potential', 'customer', 'market', 'elements', 'include', 'pricing', 'sales']
['approval', 'philippines', 'scientist', 'december', 'base', 'world', 'health', 'organization']
['therefore', 'important', 'master', 'confluence', 'discipline', 'innovation']
[]
[]
['instance', 'provide', 'individual', 'potentially', 'valuable', 'insight', 'probability']
[]
['pro

In [15]:
data_words_nostops2 = remove_stopwords(text_data2)
data_words_bigrams2 = make_bigrams(data_words_nostops2)
data_lemmatized2 = lemmatization(data_words_bigrams2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

id2word2 = corpora.Dictionary(data_lemmatized2)
# Create Corpus
texts2 = data_lemmatized2
# Term Document Frequency
corpus2 = [id2word.doc2bow(text) for text in texts2]
# View
print(corpus2[:10])

[[(66, 1)], [(33, 1), (45, 1)], [(33, 1), (45, 1), (165, 1)], [(39, 1)], [], [], [], [], [], []]


In [50]:
lda_model2 = gensim.models.LdaMulticore(corpus=corpus2,
                                       id2word=id2word2,
                                       num_topics=4, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [51]:
pprint(lda_model2.print_topics())
doc2_lda = lda_model2[corpus]

[(0,
  '0.029*"challenge" + 0.016*"appropriate" + 0.016*"directly" + 0.016*"time" + '
  '0.016*"people" + 0.015*"engineering" + 0.013*"live" + 0.004*"right" + '
  '0.003*"business" + 0.003*"infectious"'),
 (1,
  '0.026*"monetize" + 0.026*"source" + 0.026*"likely" + 0.015*"product" + '
  '0.015*"proposition" + 0.015*"scientist" + 0.015*"particular" + '
  '0.015*"production" + 0.015*"societal" + 0.015*"protein"'),
 (2,
  '0.043*"live" + 0.031*"analysis" + 0.031*"utilize" + 0.031*"identify" + '
  '0.031*"scientific" + 0.022*"sale" + 0.021*"discipline" + 0.012*"instance" + '
  '0.012*"enable" + 0.012*"change"'),
 (3,
  '0.086*"business" + 0.033*"right" + 0.024*"utilize" + 0.023*"economic" + '
  '0.023*"infectious" + 0.013*"thrive" + 0.013*"nuclear" + 0.013*"application" '
  '+ 0.013*"community" + 0.013*"effect"')]


In [52]:
#bio

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model2, corpus2, id2word2)
LDAvis_prepared

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.055842 -0.031719       1        1  35.090149
3     -0.054262 -0.033051       2        1  30.907515
1      0.000224  0.046704       3        1  19.605982
0     -0.001805  0.018066       4        1  14.396352, topic_info=    Category      Freq         Term     Total  loglift  logprob
33   Default  2.000000     business  2.000000  30.0000  30.0000
17   Default  1.000000         live  1.000000  29.0000  29.0000
155  Default  0.000000    challenge  0.000000  28.0000  28.0000
45   Default  0.000000        right  0.000000  27.0000  27.0000
21   Default  0.000000       source  0.000000  26.0000  26.0000
191  Default  0.000000       likely  0.000000  25.0000  25.0000
108  Default  1.000000     identify  1.000000  24.0000  24.0000
165  Default  1.000000   scientific  1.000000  23.0000  23.0000
187  Default  0.000000     monetize  0.000000  22.0000  22.0000
146  Default  0.000000     economic  0.000000  21.0000  21.0000
196  Default  0.000000   infectious  0.000000  20.0000  20.0000
169  Default  0.000000  appropriate  0.000000  19.0000  19.0000
64   Default  0.000000     directly  0.000000  18.0000  18.0000
161  Default  0.000000         time  0.000000  17.0000  17.0000
83   Default  0.000000       people  0.000000  16.0000  16.0000
96   Default  1.000000      utilize  1.000000  15.0000  15.0000
76   Default  1.000000     analysis  1.000000  14.0000  14.0000
174  Default  0.000000  engineering  0.000000  13.0000  13.0000
46   Default  0.000000         sale  0.000000  12.0000  12.0000
49   Default  0.000000   discipline  0.000000  11.0000  11.0000
37   Default  0.000000  proposition  0.000000  10.0000  10.0000
47   Default  0.000000    scientist  0.000000   9.0000   9.0000
66   Default  0.000000   particular  0.000000   8.0000   8.0000
13   Default  0.000000     societal  0.000000   7.0000   7.0000
185  Default  0.000000   production  0.000000   6.0000   6.0000
60   Default  0.000000      protein  0.000000   5.0000   5.0000
75   Default  0.000000      advance  0.000000   4.0000   4.0000
34   Default  0.000000       market  0.000000   3.0000   3.0000
105  Default  0.000000      product  0.000000   2.0000   2.0000
68   Default  0.000000      nuclear  0.000000   1.0000   1.0000
..       ...       ...          ...       ...      ...      ...
151   Topic4  0.037094       effort  0.217152   0.1711  -5.7505
153   Topic4  0.037094    repurpose  0.217152   0.1711  -5.7505
45    Topic4  0.043882        right  0.994816  -1.1829  -5.5825
282   Topic4  0.037094        truly  0.217152   0.1711  -5.7505
117   Topic4  0.037094    biohacker  0.217152   0.1711  -5.7505
196   Topic4  0.039592   infectious  0.738066  -0.9872  -5.6854
120   Topic4  0.037094       crispr  0.217152   0.1711  -5.7505
4     Topic4  0.037094      partner  0.217152   0.1711  -5.7505
122   Topic4  0.037094    synthetic  0.217152   0.1711  -5.7505
146   Topic4  0.038657     economic  0.738778  -1.0121  -5.7093
33    Topic4  0.039797     business  2.305493  -2.1211  -5.6802
108   Topic4  0.038682     identify  1.036963  -1.3505  -5.7086
96    Topic4  0.037262      utilize  1.560954  -1.7969  -5.7460
76    Topic4  0.037233     analysis  1.299660  -1.6145  -5.7468
46    Topic4  0.037227         sale  0.764425  -1.0839  -5.7470
39    Topic4  0.037209        value  0.490712  -0.6411  -5.7475
187   Topic4  0.037203     monetize  0.860313  -1.2027  -5.7476
177   Topic4  0.037200     perceive  0.490732  -0.6414  -5.7477
18    Topic4  0.037198     organism  0.490718  -0.6414  -5.7477
165   Topic4  0.037197   scientific  1.035810  -1.3885  -5.7478
21    Topic4  0.037196       source  0.587365  -0.8212  -5.7478
34    Topic4  0.037193       market  0.402277  -0.4428  -5.7479
131   Topic4  0.037187      example  0.490732  -0.6417  -5.7481
75    Topic4  0.037187      advance  0.402280  -0.4430  -5.7481
13    Topic4  0.037186     societal  0.402267  -0

In [19]:
#connect
text_data3=[]
with open("/Users/shiqinchoo/Desktop/combined/connect.txt","r") as h:
    for line in h:
        tokens = prepare_text_for_lda(line)
        if random.random() > .95:
            print(tokens)
            text_data3.append(tokens)
            
data_words_nostops3 = remove_stopwords(text_data3)
data_words_bigrams3 = make_bigrams(data_words_nostops3)
data_lemmatized3 = lemmatization(data_words_bigrams3, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

id2word3 = corpora.Dictionary(data_lemmatized3)
    # Create Corpus
texts3 = data_lemmatized3
    # Term Document Frequency
corpus3 = [id2word.doc2bow(text) for text in texts3]
    # View
print(corpus[:10])

['alexandre', 'ménard']
[]
['value', 'stake']
[]
['increase', 'global', 'trillion', 'trillion', 'connectivity', 'enable', 'innovation']
[]
['global', 'addressable', 'population', 'remain', 'widen', 'pattern', 'deployment']
['cover', 'approximately', 'percent', 'services', 'extent', 'processing']
['global', 'population', 'advance', 'connectivity', 'migrate', 'provider', 'continue']
[]
[]
['implement', 'system', 'result', 'productivity', 'gain']
['could', 'another', 'trillion', 'trillion', 'beyond', 'economic', 'value']
[]
['connectivity', 'enable', 'business', 'decade', 'well.5', 'enhance']
['automotive', 'industry', 'heart', 'mobility', 'broad', 'concept', 'include']
[]
['case', 'retail', 'could', 'boost', 'billion', 'billion']
['connect', 'world']
['vision', 'quality']
['frictionless']
['case', 'capability', 'relate', 'enhance', 'broadband', 'mission-']
['industry', 'relevance', 'instance', 'network', 'experience', 'future', 'development']
['network', 'sensor', 'enable', 'inventory', 

[[(0, 1), (1, 1), (2, 1), (3, 1)], [], [(4, 1), (5, 1)], [(6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)], [], [], [(12, 1), (13, 1), (14, 1), (15, 1), (16, 1)], [], [], [(17, 1)]]


In [31]:
lda_model3 = gensim.models.LdaMulticore(corpus=corpus3,
                                        id2word=id2word3,
                                        num_topics=4,
                                        random_state=100,
                                        chunksize=100,
                                        passes=10,
                                        per_word_topics=True)

In [32]:
pprint(lda_model3.print_topics())
doc3_lda = lda_model3[corpus]

[(0,
  '0.034*"manufacturing" + 0.027*"extremely" + 0.027*"coverage" + '
  '0.026*"market" + 0.021*"experience" + 0.014*"mobile" + '
  '0.014*"substantially" + 0.014*"cover" + 0.014*"vehicle" + 0.014*"third"'),
 (1,
  '0.036*"advance" + 0.036*"promise" + 0.029*"vision" + 0.022*"additional" + '
  '0.015*"barrier" + 0.009*"pattern" + 0.009*"adoption" + 0.009*"boost" + '
  '0.009*"value" + 0.009*"tracking"'),
 (2,
  '0.033*"well" + 0.022*"relate" + 0.021*"collaborate" + 0.015*"capability" + '
  '0.015*"limit" + 0.015*"economic" + 0.015*"deployment" + 0.014*"extent" + '
  '0.014*"frontier" + 0.014*"geography"'),
 (3,
  '0.083*"technology" + 0.032*"widen" + 0.029*"well" + 0.022*"broadband" + '
  '0.018*"imply" + 0.018*"security" + 0.018*"automate" + 0.017*"manufacturing" '
  '+ 0.012*"area" + 0.012*"demand"')]


In [33]:
#connect
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model3, corpus3, id2word3)
LDAvis_prepared

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.077339  0.017003       1        1  34.461777
0      0.040637  0.039708       2        1  23.860920
2      0.004158 -0.054498       3        1  22.455606
1      0.032544 -0.002213       4        1  19.221691, topic_info=    Category      Freq           Term     Total  loglift  logprob
145  Default  4.000000     technology  4.000000  30.0000  30.0000
18   Default  1.000000        advance  1.000000  29.0000  29.0000
69   Default  1.000000        promise  1.000000  28.0000  28.0000
44   Default  1.000000         vision  1.000000  27.0000  27.0000
33   Default  2.000000           well  2.000000  26.0000  26.0000
169  Default  2.000000  manufacturing  2.000000  25.0000  25.0000
177  Default  1.000000      extremely  1.000000  24.0000  24.0000
120  Default  1.000000       coverage  1.000000  23.0000  23.0000
11   Default  2.000000          widen  2.000000  22.0000  22.0000
108  Default  1.000000         market  1.000000  21.0000  21.0000
76   Default  0.000000    collaborate  0.000000  20.0000  20.0000
49   Default  1.000000         relate  1.000000  19.0000  19.0000
119  Default  1.000000     additional  1.000000  18.0000  18.0000
46   Default  1.000000      broadband  1.000000  17.0000  17.0000
71   Default  0.000000        barrier  0.000000  16.0000  16.0000
194  Default  1.000000          imply  1.000000  15.0000  15.0000
75   Default  1.000000       security  1.000000  14.0000  14.0000
118  Default  0.000000          limit  0.000000  13.0000  13.0000
28   Default  0.000000       economic  0.000000  12.0000  12.0000
7    Default  0.000000     deployment  0.000000  11.0000  11.0000
14   Default  0.000000         extent  0.000000  10.0000  10.0000
165  Default  1.000000       automate  1.000000   9.0000   9.0000
163  Default  0.000000       frontier  0.000000   8.0000   8.0000
147  Default  0.000000      geography  0.000000   7.0000   7.0000
51   Default  1.000000     experience  1.000000   6.0000   6.0000
125  Default  0.000000  substantially  0.000000   5.0000   5.0000
13   Default  0.000000          cover  0.000000   4.0000   4.0000
176  Default  0.000000        vehicle  0.000000   3.0000   3.0000
137  Default  0.000000          third  0.000000   2.0000   2.0000
161  Default  0.000000         mobile  0.000000   1.0000   1.0000
..       ...       ...            ...       ...      ...      ...
203   Topic4  0.053870       forecast  0.251789   0.1071  -6.3658
204   Topic4  0.053870         update  0.251789   0.1071  -6.3658
205   Topic4  0.053870         visual  0.251789   0.1071  -6.3658
206   Topic4  0.053870   continuation  0.251789   0.1071  -6.3658
207   Topic4  0.053870          first  0.251789   0.1071  -6.3658
208   Topic4  0.053870          trend  0.251789   0.1071  -6.3658
209   Topic4  0.053870        content  0.251789   0.1071  -6.3658
51    Topic4  0.256830     experience  1.516884  -0.1269  -4.8040
210   Topic4  0.053870     definition  0.251789   0.1071  -6.3658
189   Topic4  0.053870      portfolio  0.251789   0.1071  -6.3658
33    Topic4  0.095788           well  2.973068  -1.7861  -5.7902
476   Topic4  0.053870      colleague  0.251789   0.1071  -6.3658
165   Topic4  0.058232       automate  1.179181  -1.3590  -6.2879
120   Topic4  0.055205       coverage  1.235370  -1.4589  -6.3413
188   Topic4  0.053870         player  0.251789   0.1071  -6.3658
145   Topic4  0.055088     technology  4.887255  -2.8363  -6.3434
11    Topic4  0.054377          widen  2.084317  -1.9971  -6.3564
49    Topic4  0.054355         relate  1.204373  -1.4491  -6.3568
169   Topic4  0.054258  manufacturing  2.402169  -2.1413  -6.3586
76    Topic4  0.054217    collaborate  0.961008  -1.2259  -6.3594
96    Topic4  0.054173           area  0.871077  -1.1284  -6.3602
108   Topic4  0.054149         market  1.233255  -1.4765  -6.3606
176   Topic4  0.054125        vehicle  0.742971  -0.9702  -6.3611
75 

In [23]:
def to_lda(filename, number):
    text_data=[]
    with open(filename,"r") as h:
        for line in h:
            tokens = prepare_text_for_lda(line)
            if random.random() > .95:
                print(tokens)
                text_data.append(tokens)
            
    data_words_nostops = remove_stopwords(text_data)
    data_words_bigrams = make_bigrams(data_words_nostops)
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    id2word = corpora.Dictionary(data_lemmatized)
    # Create Corpus
    texts = data_lemmatized
    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    # View
    print(corpus[:10])
    
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                        id2word=id2word,
                                        num_topics=number,
                                        random_state=100,
                                        chunksize=100,
                                        passes=10,
                                        per_word_topics=True)
    
    
    pprint(lda_model.print_topics())
    doc_lda = lda_model[corpus]
    
    return lda_model, corpus, id2word
    
  

In [24]:
trade_lda = to_lda("/Users/shiqinchoo/Desktop/combined/trade.txt",10)

[]
[]
['james', 'manyika', 'chairman', 'base', 'francisco', 'jonathan', 'woetzel']
['academic', 'adviser', 'challenge', 'thinking', 'add', 'insight', 'extend', 'sincere']
['graduate', 'institute', 'geneva', 'carol', 'corrado', 'conference', 'board', 'jonathan', 'haskel']
[]
['francisco']
[]
['senior', 'partner', 'mckinsey', 'company']
[]
['second', 'cross', 'border', 'services', 'growing']
['company', 'localize', 'supply', 'chains', 'better']
['aspect', 'value', 'add', 'services', 'contribute']
[]
[]
['affect']
[]
['fast', 'growth', 'services', 'trade']
['priority', 'global', 'company', 'challenge', 'country']
[]
['measure', 'services', 'already', 'constitute', 'value', 'global', 'trade', 'good']
[]
['digital', 'globalization', 'global', 'flow', 'mckinsey', 'global', 'institute', 'february', 'foreign']
['economy', 'mckinsey', 'global', 'institute', 'april']
['although', 'value', 'chains', 'synonymous', 'mind', 'globalization']
['distribute', 'around', 'world', 'country', 'include', 'de

['blockchain', 'logistics', 'efficient', 'automate', 'payment', 'blockchain', 'base', 'smart', 'contract']
['buyer', 'sellers.97']
[]
['choudary', 'platform', 'revolution', 'network', 'market', 'transform', 'economy']
['function', 'match', 'customer', 'supplier', 'facilitate', 'ordering', 'accept', 'payment', 'coordinate']
['customs', 'processing', 'trade', 'facilitation', 'agreement', 'effect']
['standing', 'most.100', 'continuation', 'small', 'parcel', 'trade', 'fuel', 'commerce']
[]
['connect', 'freelancer', 'around', 'world', 'client', 'services', 'coding']
['internet', 'things', 'blockchain', 'automation', 'improve', 'logistics']
['improve', 'logistics', 'contribute', 'increase', 'trade.102', 'today', 'range']
['monitor', 'location', 'temperature', 'humidity', 'international', 'shipment']
[]
[]
['world']
[]
['benefit', 'entire', 'industry', 'national', 'economy', 'ramification']
['accommodation', 'services', 'transportation', 'warehousing', 'exhibit']
['company', 'consider', 'rang

[[], [], [], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [], [], [], [], [(8, 1), (9, 1)], []]
[(0,
  '0.077*"service" + 0.048*"trade" + 0.023*"global" + 0.020*"market" + '
  '0.019*"growth" + 0.018*"include" + 0.018*"digital" + 0.016*"economy" + '
  '0.015*"technology" + 0.011*"logistic"'),
 (1,
  '0.046*"country" + 0.019*"partner" + 0.019*"activity" + 0.019*"create" + '
  '0.014*"great" + 0.014*"product" + 0.014*"production" + 0.014*"example" + '
  '0.014*"account" + 0.014*"strategy"'),
 (2,
  '0.082*"value" + 0.068*"chain" + 0.026*"trade" + 0.021*"future" + '
  '0.021*"share" + 0.016*"economy" + 0.016*"global" + 0.014*"price" + '
  '0.014*"develop" + 0.012*"big"'),
 (3,
  '0.054*"global" + 0.026*"model" + 0.026*"competition" + 0.022*"operation" + '
  '0.022*"impact" + 0.022*"innovator" + 0.019*"platform" + 0.015*"source" + '
  '0.015*"revenue" + 0.015*"develop"'),
 (4,
  '0.044*"output" + 0.036*"gross" + 0.024*"flow" + 0.023*"trade" + '
  '0.021*"good" + 0.017*

In [25]:
trade_lda[0]

In [26]:
#trade

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(trade_lda[0], trade_lda[1], trade_lda[2])
LDAvis_prepared
    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.153300  0.011009       1        1  17.331667
2     -0.123935 -0.135351       2        1  14.371692
6     -0.048564  0.079931       3        1  12.608057
0     -0.052628  0.175950       4        1  12.517298
8      0.037827 -0.014057       5        1   9.496928
3      0.131469  0.029603       6        1   7.572042
4     -0.031266 -0.048138       7        1   7.215250
5      0.063202 -0.061052       8        1   6.730079
9      0.100153 -0.001413       9        1   6.500713
1      0.077041 -0.036481      10        1   5.656276, topic_info=    Category       Freq         Term      Total  loglift  logprob
20   Default  20.000000        value  20.000000  30.0000  30.0000
14   Default  19.000000        chain  19.000000  29.0000  29.0000
12   Default  22.000000      service  22.000000  28.0000  28.0000
25   Default  21.000000      country  21.000000  27.0000  27.0000
44   Default   7.000000       output   7.000000  26.0000  26.0000
518  Default   5.000000       worker   5.000000  25.0000  25.0000
26   Default  20.000000       global  20.000000  24.0000  24.0000
24   Default  32.000000        trade  32.000000  23.0000  23.0000
572  Default   4.000000        gross   4.000000  22.0000  22.0000
256  Default   4.000000      process   4.000000  21.0000  21.0000
59   Default   7.000000      product   7.000000  20.0000  20.0000
51   Default   7.000000        labor   7.000000  19.0000  19.0000
34   Default  12.000000      develop  12.000000  18.0000  18.0000
29   Default   9.000000         good   9.000000  17.0000  17.0000
183  Default   6.000000         flow   6.000000  16.0000  16.0000
173  Default   4.000000  competition   4.000000  15.0000  15.0000
81   Default   9.000000      percent   9.000000  14.0000  14.0000
85   Default   4.000000       impact   4.000000  13.0000  13.0000
107  Default   3.000000        model   3.000000  12.0000  12.0000
48   Default   8.000000        share   8.000000  11.0000  11.0000
201  Default   4.000000       change   4.000000  10.0000  10.0000
483  Default   2.000000      exhibit   2.000000   9.0000   9.0000
145  Default   5.000000      advance   5.000000   8.0000   8.0000
109  Default   4.000000       future   4.000000   7.0000   7.0000
118  Default   4.000000     customer   4.000000   6.0000   6.0000
120  Default   3.000000    operation   3.000000   5.0000   5.0000
38   Default   9.000000        world   9.000000   4.0000   4.0000
194  Default   3.000000     activity   3.000000   3.0000   3.0000
531  Default   2.000000    innovator   2.000000   2.0000   2.0000
486  Default   3.000000    turbulent   3.000000   1.0000   1.0000
..       ...        ...          ...        ...      ...      ...
312  Topic10   1.056377  maintenance   2.092900   2.1887  -4.2413
367  Topic10   1.056376       reduce   2.092901   2.1887  -4.2413
8    Topic10   1.404630      partner   3.221425   2.0424  -3.9564
288  Topic10   0.409263       adjust   1.007653   1.9714  -5.1896
291  Topic10   0.409263       profit   1.007653   1.9714  -5.1896
379  Topic10   0.409263        match   1.007653   1.9714  -5.1896
376  Topic10   0.409263   coordinate   1.007653   1.9714  -5.1896
375  Topic10   0.409263       accept   1.007653   1.9714  -5.1896
378  Topic10   0.409263     function   1.007653   1.9714  -5.1896
90   Topic10   0.409262        worry   1.007653   1.9714  -5.1896
86   Topic10   0.409262       nearly   1.007653   1.9714  -5.1896
88   Topic10   0.409262  substantial   1.007653   1.9714  -5.1896
89   Topic10   0.409263       tariff   1.007653   1.9714  -5.1896
87   Topic10   0.409263   respondent   1.007653   1.9714  -5.1896
122  Topic10   0.409241       action   1.007631   1.9714  -5.1896
352  Topic10   0.408927         help   1.007509   1.9707  -5.1904
194  Topic10   1.394505     activity   3.502564   1.9514  -3.9636
560  Topic10   1.056369       sector   2.730468   1.9228  -4.2413
360  Topi

In [39]:
jobs_lda = to_lda("/Users/shiqinchoo/Desktop/combined/jobs.txt",5)


['provide', 'leaders', 'commercial', 'public', 'social', 'sector', 'fact']
['employ', 'analytical', 'tool', 'economics', 'insight', 'business']
['leaders', 'micro', 'macro', 'methodology', 'examine', 'microeconomic']
['inequality', 'productivity', 'puzzle', 'economic', 'benefit', 'tackle', 'gender']
['download', 'report', 'please', 'visit']
[]
[]
['include', 'economy', 'potential', 'impact', 'global', 'economy']
['digitization', 'automation', 'robotics', 'artificial', 'intelligence']
['technology', 'american', 'economy', 'report', 'national', 'commission', 'technology', 'automation']
['nielsen', 'narasimhan', 'seshadri', 'raman', 'sharma', 'vivien', 'singer', 'rachel', 'valentino']
['publicity', 'digital', 'editor', 'lauren', 'meling', 'provide', 'support', 'online', 'publication', 'social']
['senior', 'partner', 'mckinsey', 'company']
['james', 'manyika']
['capability', 'skill', 'relatively', 'automate']
['displacement', 'worker', 'additional', 'investment']
['population']
['future', 

['exact', 'vocational', 'enrollment', 'rates', 'available', 'national', 'center', 'educational', 'statistics', 'million', 'student']
['vocational', 'education', 'training', 'germany', 'strength', 'challenge', 'recommendation']
[]
['provide', 'student', 'skill', 'necessary', 'dynamic', 'technology-', 'increasingly']
[]
[]
['address', 'challenge']
['fewer', 'people', 'switching', 'jobs.128', 'striking', 'example', 'unite', 'state']
[]
[]
[]
['developing', 'country', 'experience', 'massive', 'migration', 'worker', 'rural', 'area', 'urban', 'area']
['review', 'current', 'employee', 'estimate', 'million', 'individual', 'could']
['displace', 'worker']
['schedule', 'unemployed', 'independent', 'provide', 'critical', 'bridge']
['prominent', 'address', 'policy', 'recommendation', 'unite']
['employer', 'pay', 'employee', 'limited', 'number', 'hours']
['individual', 'controversy', 'impact', 'unemployment']
[]
['better', 'individual', 'orientation', 'market', 'trend', 'future', 'opportunity']
['ni

In [40]:
#jobs

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(jobs_lda[0], jobs_lda[1], jobs_lda[2])
LDAvis_prepared

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.052484 -0.091163       1        1  22.865082
0      0.105168  0.075501       2        1  21.595387
4      0.008794  0.089338       3        1  20.285055
3     -0.006725 -0.102675       4        1  18.289871
2     -0.159721  0.028999       5        1  16.964607, topic_info=    Category       Freq           Term      Total  loglift  logprob
85   Default   7.000000        service   7.000000  30.0000  30.0000
80   Default   7.000000     technology   7.000000  29.0000  29.0000
113  Default   6.000000           gain   6.000000  28.0000  28.0000
114  Default   7.000000      workforce   7.000000  27.0000  27.0000
61   Default   8.000000     transition   8.000000  26.0000  26.0000
71   Default   9.000000         market   9.000000  25.0000  25.0000
67   Default  10.000000       scenario  10.000000  24.0000  24.0000
413  Default   6.000000       training   6.000000  23.0000  23.0000
372  Default   6.000000       displace   6.000000  22.0000  22.0000
316  Default   4.000000           good   4.000000  21.0000  21.0000
108  Default   5.000000     employment   5.000000  20.0000  20.0000
315  Default   7.000000      education   7.000000  19.0000  19.0000
109  Default  11.000000         growth  11.000000  18.0000  18.0000
32   Default  19.000000     automation  19.000000  17.0000  17.0000
18   Default   7.000000       economic   7.000000  16.0000  16.0000
70   Default  14.000000          labor  14.000000  15.0000  15.0000
21   Default   8.000000   productivity   8.000000  14.0000  14.0000
29   Default   5.000000        include   5.000000  13.0000  13.0000
101  Default   3.000000        exhibit   3.000000  12.0000  12.0000
153  Default   4.000000     individual   4.000000  11.0000  11.0000
60   Default   6.000000        percent   6.000000  10.0000  10.0000
53   Default  12.000000     occupation  12.000000   9.0000   9.0000
431  Default   3.000000         switch   3.000000   8.0000   8.0000
27   Default   5.000000         global   5.000000   7.0000   7.0000
131  Default   3.000000         manage   3.000000   6.0000   6.0000
317  Default   3.000000         health   3.000000   5.0000   5.0000
441  Default   4.000000           area   4.000000   4.0000   4.0000
136  Default   6.000000          model   6.000000   3.0000   3.0000
26   Default   5.000000        economy   5.000000   2.0000   2.0000
233  Default   3.000000  technological   3.000000   1.0000   1.0000
..       ...        ...            ...        ...      ...      ...
539   Topic5   1.215510  international   1.752677   1.4081  -5.2063
523   Topic5   1.215510         unlock   1.752677   1.4081  -5.2063
540   Topic5   1.215510       tradable   1.752677   1.4081  -5.2063
554   Topic5   1.215509           give   1.752677   1.4081  -5.2063
555   Topic5   1.215509        predict   1.752677   1.4081  -5.2063
534   Topic5   1.215506         formal   1.752676   1.4081  -5.2063
488   Topic5   1.215506       schedule   1.752676   1.4081  -5.2063
533   Topic5   1.215506  differentiate   1.752676   1.4081  -5.2063
532   Topic5   1.215504     credential   1.752676   1.4081  -5.2063
464   Topic5   1.215500       customer   1.752675   1.4081  -5.2063
428   Topic5   1.215466         author   1.752667   1.4080  -5.2063
429   Topic5   1.215464       designer   1.752667   1.4080  -5.2063
601   Topic5   1.215461          index   1.752666   1.4080  -5.2063
147   Topic5   1.919307          point   2.890888   1.3644  -4.7495
29    Topic5   3.187377        include   5.280420   1.2692  -4.2423
21    Topic5   4.175807   productivity   8.352842   1.0807  -3.9721
32    Topic5   6.031194     automation  19.458950   0.6027  -3.6045
372   Topic5   2.295233       displace   6.730394   0.6982  -4.5706
305   Topic5   1.519392         driver   3.303066   0.9975  -4.9831
159   Topic5   1.745762          trend   5.511610   0.6244  -4.8443
5     Topic5   1.453418         sector   3.

In [29]:
social_lda = to_lda("/Users/shiqinchoo/Desktop/combined/social.txt",6)


[]
['research', 'mckinsey', 'company', 'leaders', 'commercial', 'public']
['methodology', 'examine', 'microeconomic', 'industry', 'trend', 'better', 'understand', 'broad']
['century']
['tilman', 'tacke', 'munich']
['abdulla', 'abdulaal', 'dubai']
['aspect', 'report', 'latest', 'publication', 'focusing', 'shifting', 'economic', 'outcome']
['director', 'jonathan', 'woetzel', 'provide', 'input', 'guidance', 'support']
['research', 'benefit', 'growing', 'various', 'aspect', 'implicit']
['source', 'research', 'throughout', 'report', 'jacob', 'hacker', 'great']
['publication', 'primarily', 'pressure', 'squeeze', 'middle', 'class', 'measuring']
['jonathan', 'woetzel']
[]
[]
['social', 'contract', 'century', 'outcome', 'worker', 'consumer']
['effects', 'force', 'economy', 'degree', 'however', 'need', 'given', 'scale']
[]
['mckinsey', 'global', 'institute']
[]
['unprecedented', 'growth', 'polarize', 'labor', 'market']
[]
['etirement', 'saving', 'retirement']
[]
[]
['getty', 'image']
['populatio

[]
['zealand']
['japan']
[]
['social', 'contract', 'century']
[]
[]
['income', 'define', 'income', 'adjust', 'inflation', 'using', 'rates', '2017=100', 'income', 'define', 'wide', 'database', 'income']
['source', 'wide', 'income', 'inequality', 'database', 'eurostat', 'world', 'mckinsey', 'global', 'institute', 'analysis']
[]
[]
['equal', 'treatment', 'permanent', 'temporary', 'worker', 'create', 'composite', 'metric', 'range']
['employment', 'protection', 'among', 'things.169', 'agreement', 'negotiate']
['scarcity', 'skill', 'rather', 'broad', 'sharing', 'gains.171']
['jimmy', 'o’donnell', 'shambaugh', 'shift', 'private', 'sector', 'union', 'participation', 'explanation']
[]
['healthcare', 'education', 'beyond', 'within', 'constraint', 'income']
['chapter', 'examine', 'individual', 'advance', 'economy', 'fare', 'consumer']
[]
['share']
['european', 'healthcare']
[]
[]
['education']
[]
['recreation']
[]
[]
['transportation']
[]
['transportation']
['around', 'world', 'billion', 'interna

['longer', 'require', 'degree', 'intesa', 'sanpaolo', 'italy', 'offering', 'collateral-']
['capacity', 'behalf', 'decline', 'household', 'saving', 'percentage']
['employer', 'south', 'korea', 'basic', 'pension', 'program', 'introduce']
['although', 'always', 'formally', 'accredit', 'provide', 'skill', 'individual', 'obtain', 'would']
['estimate', 'bottom', 'percent', 'wealth', 'distribution', 'return', 'percent', 'base']
['ireland', 'population', 'ready', 'retirement', 'mckinsey', 'company']
['financial', 'health', 'program', 'employee', 'target', 'area', 'pay']
['million', 'individual', 'country', 'facing', 'difficulty', 'position']
['facing', 'systematic', 'shortfall', 'access', 'stable', 'employment', 'affordable', 'housing']
['daria', 'solovieva', 'paypal', 'unveil', 'financial', 'health', 'program', 'employee', 'market', 'working', 'karma', 'impact']
['monzo', 'estimate', 'saving', 'challenge', 'individual']
['statistics']
['approximately', 'million', 'black', 'hispanic', 'minorit

In [30]:
#social

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(social_lda[0], social_lda[1], social_lda[2])
LDAvis_prepared

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.058375 -0.117255       1        1  21.798019
1     -0.122471  0.129521       2        1  16.152414
4     -0.069278 -0.054374       3        1  15.877896
5      0.083586 -0.027407       4        1  15.516345
2      0.116484  0.061147       5        1  15.376270
3      0.050054  0.008368       6        1  15.279053, topic_info=    Category       Freq            Term      Total  loglift  logprob
57   Default  15.000000         country  15.000000  30.0000  30.0000
58   Default  11.000000          income  11.000000  29.0000  29.0000
60   Default  18.000000         pension  18.000000  28.0000  28.0000
150  Default  10.000000           unite  10.000000  27.0000  27.0000
108  Default   9.000000       household   9.000000  26.0000  26.0000
264  Default   8.000000           point   8.000000  25.0000  25.0000
30   Default   7.000000          source   7.000000  24.0000  24.0000
59   Default   9.000000           level   9.000000  23.0000  23.0000
143  Default   6.000000         private   6.000000  22.0000  22.0000
62   Default   8.000000      population   8.000000  21.0000  21.0000
149  Default   6.000000           state   6.000000  20.0000  20.0000
263  Default   7.000000      percentage   7.000000  19.0000  19.0000
343  Default   7.000000       education   7.000000  18.0000  18.0000
105  Default   7.000000      employment   7.000000  17.0000  17.0000
2    Default  10.000000          public  10.000000  16.0000  16.0000
109  Default  14.000000         percent  14.000000  15.0000  15.0000
54   Default   5.000000      retirement   5.000000  14.0000  14.0000
11   Default   4.000000         century   4.000000  13.0000  13.0000
554  Default   4.000000           leave   4.000000  12.0000  12.0000
556  Default   4.000000        parental   4.000000  11.0000  11.0000
95   Default   6.000000          access   6.000000  10.0000  10.0000
133  Default   7.000000         average   7.000000   9.0000   9.0000
148  Default  16.000000      individual  16.000000   8.0000   8.0000
48   Default   5.000000          growth   5.000000   7.0000   7.0000
39   Default  15.000000          worker  15.000000   6.0000   6.0000
253  Default   4.000000          sample   4.000000   5.0000   5.0000
209  Default   4.000000           index   4.000000   4.0000   4.0000
125  Default   4.000000       challenge   4.000000   3.0000   3.0000
138  Default   5.000000     replacement   5.000000   2.0000   2.0000
213  Default   3.000000      inequality   3.000000   1.0000   1.0000
..       ...        ...             ...        ...      ...      ...
475   Topic6   0.969194        quintile   1.553957   1.4066  -5.4745
594   Topic6   0.969194           score   1.553957   1.4066  -5.4745
560   Topic6   0.969191        moderate   1.553956   1.4066  -5.4745
131   Topic6   2.372610          health   4.050707   1.3438  -4.5792
130   Topic6   3.102450          global   5.474557   1.3108  -4.3110
410   Topic6   1.822371         account   3.260821   1.2968  -4.8430
136   Topic6   1.524605      proportion   2.663232   1.3209  -5.0214
347   Topic6   1.779624  transportation   3.202826   1.2911  -4.8668
210   Topic6   1.753380            well   3.220163   1.2708  -4.8816
257   Topic6   2.634939            work   5.837806   1.0832  -4.4743
309   Topic6   1.524448        relative   2.962383   1.2143  -5.0215
127   Topic6   1.823895      investment   3.847769   1.1322  -4.8422
109   Topic6   4.497695         percent  14.302572   0.7218  -3.9396
155   Topic6   2.678081       financial   6.804663   0.9462  -4.4581
167   Topic6   2.380355           asset   6.060153   0.9442  -4.5759
62    Topic6   2.634372      population   8.574834   0.6985  -4.4745
48    Topic6   2.090517          growth   5.982392   0.8273  -4.7058
273   Topic6   1.823479        employee   4.743983   0.9226  -4.8424
232   Topic6   1.225867    productivity   2.365467   1.2214  -5.2395
121   